In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
from tensorflow.keras.models import load_model
import torch.nn.functional as F
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input





In [2]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Resize to fit model input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

In [3]:
# Load the X-ray training dataset
train_dataset_path = './Data/train'  # Replace with your train dataset path
train_dataset = ImageFolder(root=train_dataset_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)


In [4]:
# Load the X-ray testing dataset
test_dataset_path = './Data/test'  # Replace with your test dataset path
test_dataset = ImageFolder(root=test_dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [5]:
# # Load saved models
# def load_model(model_class ,path):
#     model = model_class()
#     model = torch.load(path)
#     return model

# # Load AlexNet model
# alexnet_model = load_model(models.alexnet, './models/alexnet_model.pth')

# # Load ResNeXt model
# resnext_model = load_model('./models/resnext_model.pth')

# # Load DenseNet model
# densenet_model = load_model('./models/densenet_model.pth')

alexnet_model = load_model('./models/alexnet_model.h5')
print("AlexNet Model Loaded")

densenet_model = load_model('./models/densenet_model.h5')
print("DenseNet Model Loaded")

resnext_model = load_model('./models/alexnet_model.h5')
print("resnext Model Loaded")



AlexNet Model Loaded
DenseNet Model Loaded
resnext Model Loaded


In [6]:
# Define the ensemble model
class Ensemble(nn.Module):
    def __init__(self, model1, model2, model3):
        super(Ensemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        
    def forward(self, x):
        # Get predictions from each model
        output1 = self.model1(x)
        output2 = self.model2(x)
        output3 = self.model3(x)
        
        # Average the predictions
        ensemble_output = (output1 + output2 + output3) / 3
        
        return ensemble_output


In [7]:
# Create an instance of the ensemble model
ensemble_model = Ensemble(alexnet_model, resnext_model, densenet_model)

# Move the ensemble model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model.to(device)

Ensemble()

In [8]:
# Set the ensemble model to evaluation mode
ensemble_model.eval()

Ensemble()

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_predictions(model, data_loader):
    ensemble_model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in data_loader:
            images = images.to(device)
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            predictions.append(probabilities.cpu().numpy())
    return np.concatenate(predictions)


In [11]:

# Get predictions from each individual model
alexnet_predictions = get_predictions(alexnet_model, test_loader)
resnext_predictions = get_predictions(resnext_model, test_loader)
densenet_predictions = get_predictions(densenet_model, test_loader)

# Combine predictions using averaging
ensemble_predictions = (alexnet_predictions + resnext_predictions + densenet_predictions) / 3

# Optionally, convert ensemble predictions to labels using argmax
ensemble_labels = np.argmax(ensemble_predictions, axis=1)

AttributeError: 'torch.Size' object has no attribute 'as_list'

In [ ]:
# # def get_accuracy(model, data_loader):
# #     correct = 0
# #     total = 0
# #     with torch.no_grad():
# #         for images, labels in data_loader:
# #             images = images.to(device)
# #             labels = labels.to(device)
# #             outputs = model(images).cpu().numpy()  # Convert outputs tensor to NumPy array
# #             predicted = np.argmax(outputs.cpu().numpy(), axis=1)  # Convert model outputs to NumPy array
# #             total += labels.size(0)
# #             correct += np.sum(predicted == labels.cpu().numpy())  # Convert labels to NumPy array and compare
# #     return correct / total

# # # Calculate accuracy for the ensemble model
# # ensemble_accuracy = get_accuracy(ensemble_model, test_loader)
# # print("Ensemble Model Accuracy:", ensemble_accuracy)


def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Calculate accuracy for the ensemble model
ensemble_accuracy = get_accuracy(ensemble_model, test_loader)
print("Ensemble Model Accuracy:", ensemble_accuracy)



AttributeError: 'torch.Size' object has no attribute 'as_list'

In [ ]:
# # Save the ensemble model
# torch.save(ensemble_model.state_dict(), './models/ensemble_model.pth')


In [ ]:
# Define a function to convert PyTorch model to Keras model
def convert_to_keras(ensemble_model):
    # Define input layer
    inputs = Input(shape=(227, 227, 3))  # Assuming input shape is (227, 227, 3)
    
    # Define a Lambda layer to convert input tensor to PyTorch tensor
    pytorch_lambda = Lambda(lambda x: tf.py_function(ensemble_model, [x], tf.float32))(inputs)
    
    # Define the Keras model
    keras_model = Model(inputs=inputs, outputs=pytorch_lambda)
    
    return keras_model

# Convert the ensemble model to a Keras model
keras_ensemble_model = convert_to_keras(ensemble_model)

# Save the Keras ensemble model to a .h5 file
keras_ensemble_model.save('./models/ensemble_model.h5')

c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


AttributeError: 'Ensemble' object has no attribute '__name__'